# IMPORT DATA, PACKAGES, AND FUNCTIONS

## import the relevant functions and packages

In [1]:
import pandas as pd
import numpy as np
from test_case_generator_functions import alphabet, num_of_alphabet, rand_letter

# TEST CASE TYPE REFERENCE INFORMATION

In [2]:
uid = 'UID-309' 
theme = 'Names where name parts are Modified'
category = 'Typos'
sub_category = '2 Typos same name part - Adjacent'
entity_type = 'Individual'

## download the OFAC list from the web

In [3]:
ofac_list_download = pd.read_csv('https://www.treasury.gov/ofac/downloads/sdn.csv', header=None)
ofac_list_download

,0,1,2,3,4,5,6,7,8,9,10,11
0,36,AEROCARIBBEAN AIRLINES,-0-,CUBA,-0-,-0-,-0-,-0-,-0-,-0-,-0-,-0-
1,173,"ANGLO-CARIBBEAN CO., LTD.",-0-,CUBA,-0-,-0-,-0-,-0-,-0-,-0-,-0-,-0-
2,306,BANCO NACIONAL DE CUBA,-0-,CUBA,-0-,-0-,-0-,-0-,-0-,-0-,-0-,a.k.a. 'BNC'.
3,424,BOUTIQUE LA MAISON,-0-,CUBA,-0-,-0-,-0-,-0-,-0-,-0-,-0-,-0-
4,475,CASA DE CUBA,-0-,CUBA,-0-,-0-,-0-,-0-,-0-,-0-,-0-,-0-
...,...,...,...,...,...,...,...,...,...,...,...,...
10892,39438,"NEJAT, Hossein",individual,IRGC] [IFSR] [IRAN-HR,IRGC Brigadier General,-0-,-0-,-0-,-0-,-0-,-0-,"DOB Mar 1955 to Mar 1956; POB Shiraz, Iran; na..."
10893,39439,"SAJEDINIA, Hossein",individual,IRAN-HR,Deputy Operations Commander,-0-,-0-,-0-,-0-,-0-,-0-,"DOB 21 Mar 1962 to 20 Apr 1962; POB Isfahan, I..."
10894,39440,"RAHIMI, Hossein",individual,IRAN-HR,Police chief of Tehran,-0-,-0-,-0-,-0-,-0-,-0-,DOB Mar 1963 to Mar 1964; POB Markazi Province...
10895,39441,"JAVANI, Yadollah",individual,IRGC] [IFSR] [IRAN-HR,Deputy Political Commander,-0-,-0-,-0-,-0-,-0-,-0-,"DOB 1962; POB Isfahan, Iran; nationality Iran;..."


In [4]:
ofac_list = ofac_list_download[[0,1,2]]
ofac_list.columns = ['uid', 'name', 'entity_type']
ofac_list

,uid,name,entity_type
0,36,AEROCARIBBEAN AIRLINES,-0-
1,173,"ANGLO-CARIBBEAN CO., LTD.",-0-
2,306,BANCO NACIONAL DE CUBA,-0-
3,424,BOUTIQUE LA MAISON,-0-
4,475,CASA DE CUBA,-0-
...,...,...,...
10892,39438,"NEJAT, Hossein",individual
10893,39439,"SAJEDINIA, Hossein",individual
10894,39440,"RAHIMI, Hossein",individual
10895,39441,"JAVANI, Yadollah",individual


# FILTER FOR THE REQUIREMENTS OF THE TEST CASE TYPE

## filter for the requirements of the specific test case type

In [5]:
ofac_list_filtered = ofac_list[(ofac_list.entity_type == 'individual')] # only evaluate individuals
ofac_list_filtered

,uid,name,entity_type
53,2674,"ABBAS, Abu",individual
54,2675,"AL RAHMAN, Shaykh Umar Abd",individual
55,2676,"AL ZAWAHIRI, Dr. Ayman",individual
56,2677,"AL-ZOMOR, Abboud Abdul Latif Hassan",individual
57,2678,"AWDA, Abd Al Aziz",individual
...,...,...,...
10891,39437,"ZAREPOUR, Eisa",individual
10892,39438,"NEJAT, Hossein",individual
10893,39439,"SAJEDINIA, Hossein",individual
10894,39440,"RAHIMI, Hossein",individual


## randomly choose 10 rows

In [6]:
while True:
    ofac_list_sampled = ofac_list_filtered.sample(n = 10)
    cnt = [0] * 10
    for ind, name in enumerate(ofac_list_sampled['name']):
        split_name = name.upper().split(' ')
        
        for i in split_name:
            if (len(split_name) == 2) & (len(split_name[0]) <= 3):
                break
            elif (len(split_name) == 2) & (len(split_name[-1]) <= 3):
                break
                
            if all(j not in alphabet() for j in i):
                split_name.remove(i) # remove word with all numbers
            elif num_of_alphabet(i) > 3:
                cnt[ind] += 1
                break

    if all(i == 1 for i in cnt):
        break

ofac_list_sampled

,uid,name,entity_type
4053,18330,"KENAR, Ufuk",individual
8018,29907,"GHANNADI MARAGHEH, Mohammad",individual
10429,37389,"CHEPA, Alexey Vasilievich",individual
7470,28117,"PETROVIC, Zelimir",individual
920,8390,"NAMAN, Saalim",individual
713,8099,"RIHANI, Lotfi Ben Abdul Hamid Ben Ali",individual
9733,35716,"RUMYANTSEV, Alexander Grigoryevich",individual
3339,16151,"QABALAN, Muhammad",individual
8676,32611,"SHMANAI, Darya",individual
1540,10587,"NOOR MUHAMMAD, Abdul Majeed",individual


# CREATE THE TEST CASES

## create blank final test cases table

In [7]:
final_test_cases = pd.DataFrame(columns=['UID', 'Theme','Category','Sub-category','Entity-Type','Test Case ID' , 'OFAC List UID', 'Original Name','Test Case Name'])
final_test_cases

,UID,Theme,Category,Sub-category,Entity-Type,Test Case ID,OFAC List UID,Original Name,Test Case Name


## run loop to generate the test cases

In [8]:
for index, row in ofac_list_sampled.iterrows():
    original_name = row['name'].upper()
    split_original_name = original_name.split(' ') # split name into word
    
    final_test_name = split_original_name.copy()
    
    while True:
        replace_word = np.random.choice(split_original_name) # randomly choose word to be replaced
        if num_of_alphabet(replace_word) > 2:
            break
    
    random_list = list(range(len(replace_word))) 
    replace_word_list = list(replace_word)
    if final_test_name.index(replace_word) == 0: # avoid the at the beginning situation
        del random_list[0] 
        del random_list[-1:]
        
        rand_letter_index = np.random.choice(random_list) # randomly choose beginning letter to be replaced
    
        for i in range(2):
            while replace_word_list[rand_letter_index] == replace_word[rand_letter_index]: # rerandom if no letter has been replaced
                replace_word_list[rand_letter_index] = rand_letter() # replace letter
            rand_letter_index += 1
    
    elif final_test_name.index(replace_word) == len(final_test_name) - 1: # avoid the at the end situation
        if replace_word_list[-1] not in alphabet():
            del random_list[-2:]
        else:
            del random_list[-1]
        del random_list[:1]
    
        rand_letter_index = np.random.choice(random_list) # randomly choose beginning letter to be replaced
        
        for i in range(2):
            while replace_word_list[rand_letter_index] == replace_word[rand_letter_index]: # rerandom if no letter has been replaced
                replace_word_list[rand_letter_index] = rand_letter() # replace letter
            rand_letter_index -= 1
    
    else:
        del random_list[-1:]
        
        rand_letter_index = np.random.choice(random_list) # randomly choose beginning letter to be replaced
    
        for i in range(2):
            while replace_word_list[rand_letter_index] == replace_word[rand_letter_index]: # rerandom if no letter has been replaced
                replace_word_list[rand_letter_index] = rand_letter() # replace letter
            rand_letter_index += 1

    final_test_name[final_test_name.index(replace_word)] = ''.join(replace_word_list)
    
    final_test_name = ' '.join(final_test_name)
    
    final_test_cases.loc[len(final_test_cases)] = [uid, theme, category, sub_category, entity_type, uid + ' - ' + str(index), row['uid'], row['name'], final_test_name] # append to the dataframe

final_test_cases

,UID,Theme,Category,Sub-category,Entity-Type,Test Case ID,OFAC List UID,Original Name,Test Case Name
0,UID-309,Names where name parts are Modified,Typos,2 Typos same name part - Adjacent,Individual,UID-309 - 4053,18330,"KENAR, Ufuk","KKXAR, UFUK"
1,UID-309,Names where name parts are Modified,Typos,2 Typos same name part - Adjacent,Individual,UID-309 - 8018,29907,"GHANNADI MARAGHEH, Mohammad","GHANNADI MARAGHGJ, MOHAMMAD"
2,UID-309,Names where name parts are Modified,Typos,2 Typos same name part - Adjacent,Individual,UID-309 - 10429,37389,"CHEPA, Alexey Vasilievich","CHEDP, ALEXEY VASILIEVICH"
3,UID-309,Names where name parts are Modified,Typos,2 Typos same name part - Adjacent,Individual,UID-309 - 7470,28117,"PETROVIC, Zelimir","PETRSDIC, ZELIMIR"
4,UID-309,Names where name parts are Modified,Typos,2 Typos same name part - Adjacent,Individual,UID-309 - 920,8390,"NAMAN, Saalim","NAMAN, SAEEIM"
5,UID-309,Names where name parts are Modified,Typos,2 Typos same name part - Adjacent,Individual,UID-309 - 713,8099,"RIHANI, Lotfi Ben Abdul Hamid Ben Ali","RIHANI, LDWFI BEN ABDUL HAMID BEN ALI"
6,UID-309,Names where name parts are Modified,Typos,2 Typos same name part - Adjacent,Individual,UID-309 - 9733,35716,"RUMYANTSEV, Alexander Grigoryevich","RUMYHPTSEV, ALEXANDER GRIGORYEVICH"
7,UID-309,Names where name parts are Modified,Typos,2 Typos same name part - Adjacent,Individual,UID-309 - 3339,16151,"QABALAN, Muhammad","QABALAN, MUHFBMAD"
8,UID-309,Names where name parts are Modified,Typos,2 Typos same name part - Adjacent,Individual,UID-309 - 8676,32611,"SHMANAI, Darya","SHWMNAI, DARYA"
9,UID-309,Names where name parts are Modified,Typos,2 Typos same name part - Adjacent,Individual,UID-309 - 1540,10587,"NOOR MUHAMMAD, Abdul Majeed","NYJR MUHAMMAD, ABDUL MAJEED"
